# 🧹 Этап 3: Предобработка и Feature Engineering

## 🎯Цели этапа
* Обработка пропусков
* Обработка выбросов
* Кодирование категориальных признаков
* Feature Engineering (создание новых признаков)


## 3.1 - 3.4) 🛠️Предобработка датасета

In [4]:
import sys
import os
import pandas as pd

os.chdir('..')

sys.path.insert(0, 'src')

from preprocessing import ProprocessingData

os.chdir('notebooks')

df = pd.read_csv('../data/Churn_Modelling.csv')

preprocessor = ProprocessingData(df)
df_processed = preprocessor.preprocessing()

Пропуски до обработки:

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          1
Gender             0
Age                1
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          1
IsActiveMember     1
EstimatedSalary    0
Exited             0
dtype: int64

Заполнены пропуски в Geography значением: France
Заполнены пропуски в Age значением: 37.0
Заполнены пропуски в HasCrCard значением: 1.0
Заполнены пропуски в IsActiveMember значением: 1.0

Пропуски после обработки:
RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

Удалены столбцы: ['RowNumber', 'CustomerId', 'Surname']
Оставшееся количество признаков: 11

🎯 Создание новых признаков...
   Созданы: Is_Senior_A

In [5]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   CreditScore                    10002 non-null  int64  
 1   Gender                         10002 non-null  int64  
 2   Age                            10002 non-null  float64
 3   Tenure                         10002 non-null  int64  
 4   Balance                        10002 non-null  float64
 5   NumOfProducts                  10002 non-null  int64  
 6   HasCrCard                      10002 non-null  float64
 7   IsActiveMember                 10002 non-null  float64
 8   EstimatedSalary                10002 non-null  float64
 9   Exited                         10002 non-null  int64  
 10  Is_Senior_Active               10002 non-null  int32  
 11  Active_With_Multiple_Products  10002 non-null  int32  
 12  Value_Client                   10002 non-null 

### Сохраним предобработанный датасет

In [6]:
df_processed.to_csv('../data/bank_churn_preprocessing.csv',index=False)

### 📋 Обзор выполненных задач

На данном этапе был реализован комплексный пайплайн предобработки данных, включающий обработку пропусков, выбросов, создание новых признаков и кодирование категориальных переменных.

Вся логика обработки датасета была помещена в файл preprocessing.py

### 🔧 Реализованные методы предобработки

#### **1. Обработка пропущенных значений**
Так как у нас на весь датасет всего 4 пропуска, то пропущенные значения были обработаны следующим образом:

```python
- 'Geography': заполнение модой (France)
- 'Age', 'HasCrCard', 'IsActiveMember': заполнение медианой
```

**Результат:** Полностью устранены пропуски во всех признаках.

#### **2. Удаление технических столбцов**

В нашем датасете присутствуют столбцы, которые не несут практической пользы для дальнейших этапов, связанных с выбором и построением моделей, поэтому было сделано следующее:

```python
Удалены нефункциональные признаки:
- 'RowNumber' - порядковый номер записи
- 'CustomerId' - уникальный идентификатор клиента  
- 'Surname' - фамилия клиента
```

**Результат:** Сокращение размерности данных с 14 до 11 признаков.

#### **3. Обработка выбросов методом перцентилей**
На этапе EDA мы выяснили, что наши числовые признаки имеют не нормальное распределение, поэтому в качестве метода обработки выбросов был выбран следующий подход:
```python
- 'Age': ограничение диапазона 1-99 перцентилями
- 'Balance': каппинг сверху 99-м перцентилем
```

**Преимущество:** Устойчивость к ненормальным распределениям данных.

### 🎯 Feature Engineering

#### **Созданы новые признаки на основе EDA-инсайтов:**
В качестве обогащения признакового пространства былы созданы дополнительные признаки, основанные на этапе EDA:

| Признак | Тип | Бизнес-логика |
|---------|-----|---------------|
| `Is_Senior_Active` | Бинарный | Клиенты старше 40 лет, активные в банке |
| `Active_With_Multiple_Products` | Бинарный | Активные клиенты с 2+ продуктами |
| `Value_Client` | Бинарный | Клиенты с высоким балансом и 2+ продуктами |
| `New_HighRisk` | Бинарный | Новые клиенты (Tenur < 2) с 1 продуктом |
| `German_Female_Risk` | Бинарный | Женщины из Германии |
| `AgeGroup` | Категориальный | Возрастные группы для нелинейных зависимостей |

#### **Проверка корреляций новых признаков**
Все созданные признаки показали умеренные корреляции (< 0.5) с исходными переменными, за исключением признаков `Active_With_Multiple_Products`, `Is_Senior_Active`. В этих признаках присутствует небольшое превышение установленной планки, и составялем 0.57-0.59, поэтому данные признаки будем использовать в дальнейших этапах 

### 🔤 Кодирование категориальных признаков

#### **One-Hot Encoding:**
- `Geography` → `Geo_France`, `Geo_Germany`, `Geo_Spain`
- `AgeGroup` → `AgeGroup_18-30`, `AgeGroup_31-40`, `AgeGroup_41-50`, `AgeGroup_51-60`, `AgeGroup_60+`

#### **Label Encoding:**
- `Gender` → 0 (Female), 1 (Male)

**Оптимизация:** Все бинарные признаки конвертированы в `int32` для совместимости с ML-алгоритмами.

### 📊 Итоговые результаты предобработки

| Метрика | До обработки | После обработки |
|---------|--------------|-----------------|
| **Количество признаков** | 14 | 22 |
| **Пропущенные значения** | 4 признака | 0 |
| **Технические столбцы** | 3 | 0 |
| **Категориальные признаки** | 3 (в текстовом формате) | 8 (в числовом формате) |
| **Новые features** | 0 | 6 |

### 🎯 Ключевые преимущества реализованного подхода

1. **Data-driven feature engineering** - новые признаки основаны на статистических инсайтах из EDA
2. **Robust обработка** - методы устойчивы к выбросам и ненормальным распределениям  
3. **Профилактика мультиколлинеарности** - проверка корреляций между новыми и существующими признаками
4. **Готовность к моделированию** - все данные в числовом формате, оптимизированы для ML-алгоритмов

Данные полностью подготовлены для перехода к этапу построения подготовки данных к обучению моделей ML.

Предобработанный датасет 'bank_churn_preprocessing.csv' был сохранен в папку data. Он будет использовать в следующих этапах